In [ ]:
# -*- coding: utf-8 -*-
"""Libras em Vídeo.ipynb

In [ ]:
Automatically generated by Colaboratory.

In [ ]:
Original file is located at
    https://colab.research.google.com/drive/1A06mMyZlAx130ZWG9N00rZ0dHbfmAmps

Projeto: Libras em vídeo

 **Etapa 1 - Importando as bibliotecas**


<br>
import time<br>
import sys<br>
import cv2<br>
import numpy as np<br>
import matplotlib.pyplot as plt<br>
import zipfile<br>
from google.colab.patches import cv2_imshow<br>
cv2.__version__<br>
# **Etapa 2 - Conectando com o Drive**


<br>
from google.colab import drive<br>
drive.mount('/content/drive')<br>
pose_path = "/content/drive/My Drive/pose.zip"<br>
zip_object = zipfile.ZipFile(file=pose_path, mode="r")<br>
zip_object.extractall("./")<br>
imagens_path = "/content/drive/My Drive/imagens.zip"<br>
zip_object = zipfile.ZipFile(file=imagens_path, mode="r")<br>
zip_object.extractall("./")<br>
modulos_path = "/content/drive/My Drive/modulos.zip"<br>
zip_object = zipfile.ZipFile(file=modulos_path, mode="r")<br>
zip_object.extractall("./")<br>
zip_object.close()<br>
# Etapa 3 - Importando o módulo do Drive


<br>
sys.path.append('/content/modulos/')<br>
sys.path<br>
import extrator_POSICAO as posicao<br>
import extrator_ALTURA as altura<br>
import extrator_PROXIMIDADE as proximidade<br>
import alfabeto<br>
# **Etapa 4 - Carregando o modelo e estruturas da rede neural pré-treinada**


In [ ]:
arquivo_proto = "/content/pose/hand/pose_deploy.prototxt"
arquivo_pesos = "/content/pose/hand/pose_iter_102000.caffemodel"
numero_pontos = 22
pares_pose = [[0, 1], [1, 2], [2, 3], [3, 4], [0, 5], [5, 6], [6, 7], [7, 8], 
              [0, 9], [9, 10], [10, 11], [11, 12], [0, 13], [13, 14], [14, 15], 
              [15, 16], [0, 17], [17, 18], [18, 19], [19, 20]]

In [ ]:
letras = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'M', 'N', 'O', 'P', 
          'Q', 'R', 'S', 'T', 'U', 'V', 'W']


## **Etapa 5 - Ler o modelo carregado na Etapa 3**<br>


In [ ]:
modelo = cv2.dnn.readNetFromCaffe(arquivo_proto, arquivo_pesos)

In [ ]:
cor_pontoA, cor_pontoB, cor_linha = (14, 201, 255), (255, 0, 128), (192, 192, 192)
cor_txtponto, cor_txtinicial, cor_txtandamento = (10, 216, 245), (255, 0, 128), (54, 54, 54)

In [ ]:
tamanho_fonte, tamanho_linha, tamanho_circulo, espessura = 1, 1, 4, 2
fonte = cv2.FONT_HERSHEY_SIMPLEX


## **Etapa 6 - Carregando um video do Drive**
<br>
video = "/content/drive/My Drive/libras2.mp4"<br>
captura = cv2.VideoCapture(video)<br>
ret, frame = captura.read()<br>
ret<br>
imagem_largura = frame.shape[1]<br>
imagem_altura = frame.shape[0]<br>
proporsao = imagem_largura / imagem_altura<br>
imagem_largura, imagem_altura, proporsao<br>
# **Etapa 7 - Definir as dimensões da imagem de entrada.**


<br>
entrada_altura = 368<br>
entrada_largura = int(((proporsao * entrada_altura) * 8) // 8)<br>
entrada_largura, entrada_altura<br>
# Etapa 7 - Criando a variável para salvar os resultados no Drive


<br>
resultado = './libras.avi'<br>
gravar_video = cv2.VideoWriter(resultado, cv2.VideoWriter_fourcc(*'XVID'), 10,<br>
                              (frame.shape[1], frame.shape[0]))<br>
# **Etapa 8 - Lendo o modelo carregado na Etapa 3**


<br>
modelo = cv2.dnn.readNetFromCaffe(arquivo_proto, arquivo_pesos)<br>
# Etapa 9 - Exibindo as saídas


<br>
limite = 0.1<br>
while (cv2.waitKey(1) < 0):<br>
    t = time.time()<br>
    conectado, frame = captura.read()<br>
    frame_copia = np.copy(frame)<br>
    tamanho = cv2.resize(frame, (imagem_largura, imagem_altura))<br>
    mapa_suave = cv2.GaussianBlur(tamanho, (3, 3), 0, 0)<br>
    fundo = np.uint8(mapa_suave > limite)<br>
    <br>
    if not conectado:<br>
        cv2.waitKey()<br>
        break<br>
    entrada_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, <br>
                                         (entrada_largura, entrada_altura),<br>
                                    (0, 0, 0), swapRB=False, crop=False)<br>
    modelo.setInput(entrada_blob)<br>
    saida = modelo.forward()<br>
    pontos = []<br>
    for i in range(numero_pontos):<br>
        mapa_confianca = saida[0, i, :, :]<br>
        mapa_confianca = cv2.resize(mapa_confianca, (imagem_largura, imagem_altura))<br>
        <br>
        _, confianca, _, point = cv2.minMaxLoc(mapa_confianca)<br>
        if confianca > limite:<br>
            cv2.circle(frame_copia, (int(point[0]), int(point[1])), <br>
                       tamanho_circulo, cor_pontoA, thickness=espessura,<br>
                       lineType=cv2.FILLED)<br>
            cv2.putText(frame_copia, "{}".format(i), (int(point[0]), int(point[1])),<br>
                        fonte, .8,<br>
                        cor_txtponto, 2, lineType=cv2.LINE_AA)<br>
            pontos.append((int(point[0]), int(point[1])))<br>
        else:<br>
            pontos.append((0, 0))<br>
    for par in pares_pose:<br>
        parteA = par[0]<br>
        parteB = par[1]<br>
        if pontos[parteA] != (0, 0) and pontos[parteB] != (0, 0):<br>
            cv2.line(frame, pontos[parteA], pontos[parteB], cor_linha, <br>
                     tamanho_linha, lineType=cv2.LINE_AA)<br>
            cv2.circle(frame, pontos[parteA], tamanho_circulo, cor_pontoA,<br>
                       thickness=espessura, lineType=cv2.FILLED)<br>
            cv2.circle(frame, pontos[parteB], tamanho_circulo, cor_pontoB,<br>
                       thickness=espessura, lineType=cv2.FILLED)<br>
            cv2.line(fundo, pontos[parteA], pontos[parteB], cor_linha, <br>
                     tamanho_linha, lineType=cv2.LINE_AA)<br>
            cv2.circle(fundo, pontos[parteA], tamanho_circulo, cor_pontoA, <br>
                       thickness=espessura, lineType=cv2.FILLED)<br>
            cv2.circle(fundo, pontos[parteB], tamanho_circulo, cor_pontoB, <br>
                       thickness=espessura, lineType=cv2.FILLED)<br>
    posicao.posicoes = []<br>
    # dedo polegar<br>
    posicao.verificar_posicao_DEDOS(pontos[1:5], 'polegar', altura.verificar_altura_MAO(pontos))<br>
    # dedo indicador<br>
    posicao.verificar_posicao_DEDOS(pontos[5:9], 'indicador', altura.verificar_altura_MAO(pontos))<br>
    # dedo médio<br>
    posicao.verificar_posicao_DEDOS(pontos[9:13], 'medio', altura.verificar_altura_MAO(pontos))<br>
    # dedo anelar<br>
    posicao.verificar_posicao_DEDOS(pontos[13:17], 'anelar', altura.verificar_altura_MAO(pontos))<br>
    # dedo mínimo<br>
    posicao.verificar_posicao_DEDOS(pontos[17:21], 'minimo', altura.verificar_altura_MAO(pontos))<br>
    for i, a in enumerate(alfabeto.letras):<br>
        if proximidade.verificar_proximidade_DEDOS(pontos) == alfabeto.letras[i]:<br>
            cv2.putText(frame, 'Letra: ' + letras[i], (50, 50), fonte, 1, <br>
                        cor_txtinicial, tamanho_fonte,<br>
                        lineType=cv2.LINE_AA)<br>
        else:<br>
            cv2.putText(frame, 'Analisando', (250, 50), fonte, 1, <br>
                        cor_txtandamento, tamanho_fonte,<br>
                        lineType=cv2.LINE_AA)<br>
    <br>
    cv2_imshow(frame)<br>
    key = cv2.waitKey(1)<br>
    if key == 27:<br>
        break<br>
    print("Tempo total = {:.2f}seg".format(time.time() - t))<br>
    gravar_video.write(frame)<br>
gravar_video.release()